In [4]:
import loadexcel as lc
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from data_convert import*
from tqdm import tqdm
import feedback_option
import loaddb as ld
import os

class UnMatchError(Exception):
    def __init__(self,ErrorInfo):
        super().__init__(self) #初始化父类
        self.errorinfo=ErrorInfo
    def __str__(self):
        return self.errorinfo

class DataManager():
    
    def __init__(self,date,path):
        self._date=date
        self._path=path
        self.update_p_index()
        #读取车辆信息
        self._car_value=ld.get_car_info()
        
    def get_cars(self):
        return self._car_value
    
    def getPath(self):
        return self._path
    
    def get_data(self):
        return self._data
    
    def update_path(self,new_path):
        self._path=new_path
        self.update_p_index()
        
    def update_p_index(self):
        self.p_index=len(os.path.split(self._path)[0].split("//"))+1
        
    def load_path(self,path):
        self._path=path
        self.update_p_index()
        self.load()
    
    def get_Sum(self):
        return self._pd_sum
    
    def get_detail(self):
        return self._pd_detail
    
    def get_result(self):
        return self._pd_result
    
    #加载数据，如果有异常项，判断加载失败并重置    
    def load(self):
        result_=lc.excel2Dataframe(self._path,target_by="车号",date_=self._date)
        sum_,detail_=[],[]
        self._pd_sum=[]
        self._pd_detail=[]
        for item in result_:
            sum_+=item[0]
            detail_+=item[1]
        for item in tqdm(sum_):
            self._pd_sum.append(lc.excel2Dataframe_Sum(item["filepath"],self.p_index,item["sheetname"],"车号"))
        for item in tqdm(detail_):
            self._pd_detail.append(lc.excel2Dataframe_Detail(item["filepath"],self.p_index,item["sheetname"],"车号"))
        self._pd_sum=pd.concat(self._pd_sum)
        self._pd_detail=pd.concat(self._pd_detail)
        if self.clean_data(self._pd_sum)==-1:
            self.reset_data()
            return
        if self.clean_data(self._pd_detail)==-1:
            self.reset_data()
            return
        if self.join_data()==-1:
            self.reset_data()
            return
        print("数据加载成功！")
        
    #获取不匹配的数据    
    def get_dismatchs(self,df_source):
        #连接数据与车辆信息
        result_data=pd.merge(df_source,self._car_value,left_on='car_id',right_on='sub_car_id',how='left')
        #找出无法匹配的车辆
        outer_result=result_data[pd.isnull(result_data['sub_car_id'])]
        dismacths=outer_result[["route","team","car_id"]]
        return dismacths
    
    #过滤一些系统能够识别的数据
    def filter_data(self,df_source,dismacths):
        for i in range(len(dismacths)) :
            _car=dismacths.iloc[i]
            car_id_=_car["car_id"]
            route_=_car["route"]
            _r=self._car_value #[self._car_value["route"]==route_]
            result_r=_r[_r["sub_car_id"].str.contains(car_id_)]
            if len(result_r)==1:
                dismacths.iloc[i]["car_id"]=result_r["sub_car_id"].iloc[0]
        df_source.iloc[dismacths.index,0]=dismacths['car_id']
        return df_source
    
    #重置数据
    def reset_data(self):
        self._pd_sum=[]
        self._pd_detail=[]
        self._pd_result=[]
        
    #清理
    def clean_data(self,df_source):
        #去掉路
        remove_route=lambda x:x.split("路")[-1].strip()
        #去掉线
        remove_line=lambda x:x.split("线")[-1].strip()
        #去掉斜杆
        remove_diagonal=lambda x:x.split("/")[-1].strip()
        #将F改成正常拼写
        replaceF=lambda x:x.replace("Ｆ","F")
        #如果以F和D结尾，则补齐6位
        zfill_ = lambda x: x.zfill(6) if x.endswith("F")|x.endswith("D") else x
        #将所选列转为字符
        df_source.iloc[:,0]=df_source.iloc[:,0].apply(str)
        df_source.iloc[:,0]=df_source.iloc[:,0].apply(remove_route)
        df_source.iloc[:,0]=df_source.iloc[:,0].apply(remove_line)
        df_source.iloc[:,0]=df_source.iloc[:,0].apply(remove_diagonal)
        #将所选列转为剔除空格
        df_source.iloc[:,0]=df_source.iloc[:,0].str.strip()
        df_source.iloc[:,0]=df_source.iloc[:,0].apply(replaceF)
        df_source.iloc[:,0]=df_source.iloc[:,0].apply(zfill_)
        df_source.index=range(len(df_source))
        dismatchs=self.get_dismatchs(df_source)
        fd=self.filter_data(df_source,dismatchs)
        dismatchs=self.get_dismatchs(fd)
        if len(dismatchs)>0:
            self.print_dismatchs(dismatchs,"无法识别",'route','car_id')
            return -1
        if len(dismatchs)==0:
            
            return 0
    
    def print_dismatchs(self,dismatchs,dsc,s_route='route',s_id='car_id'):
        print("有数据无法匹配：")
        for i in range(len(dismatchs)):
            item=dismatchs.iloc[i]
            print(f"%s线路下的%s车%s"%(item[s_route],item[s_id],dsc))
        print(f"合计%d个项无法识别，请修正后重试"%(len(dismatchs)))
        
    def join_data(self):
        result_data=pd.merge(self.get_Sum(),self.get_detail(),on=['car_id','route'],how='outer')
        dismatchs=result_data[pd.isnull(result_data['work_days'])]
        flag=0
        if len(dismatchs):
            self.print_dismatchs(dismatchs,"缺汇总项")
            flag= -1
        dismatchs=result_data[pd.isnull(result_data['mileage'])]
        if len(dismatchs):
            self.print_dismatchs(dismatchs,"缺统计项")
            flag=-1
        self._pd_result=result_data
        return flag
            
            
        
        
        

In [2]:
import datetime

In [3]:
dm=DataManager(datetime.date(2019,8,1),"example_data//feedback//8月原始")

In [4]:
dm.load_path("example_data//feedback//8月原始")

100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:07<00:00,  7.56it/s]


数据加载成功！


In [5]:
result_data=dm.get_result()

In [6]:
date=datetime.date(2019,8,1)

In [7]:
result_data=pd.merge(result_data,dm.get_cars(),left_on='car_id',right_on='sub_car_id',how='left')

In [8]:
result_data["target_oil_cost"]=result_data["target_value2" if date.month in range(6,10) else "target_value1"]
result_data["target_elc_cost"]=result_data["target_value4" if date.month in range(6,10) else "target_value3"]
result_data["total_oil_target"]=result_data["mileage"]*result_data["target_oil_cost"]/100
result_data["total_elc_target"]=result_data["mileage"]*result_data["target_elc_cost"]/100

In [9]:
result_data

,car_id,mileage,oil_cost,maintain,follow,inspection,route,team_x,fix_days,stop_days,...,id,target_value1,target_value2,target_value3,target_value4,power_type,target_oil_cost,target_elc_cost,total_oil_target,total_elc_target
0,B723,6273.6,995.71,5,0.0,0.0,501,1,0,2,...,11,14,17,0,0,柴,17,0,1066.512,0.00
1,B728,3829.6,591.58,5,0.0,0.0,501,1,0,14,...,11,14,17,0,0,柴,17,0,651.032,0.00
2,B835,6925.2,980.76,0,0.0,0.0,501,1,0,1,...,11,14,17,0,0,柴,17,0,1177.284,0.00
3,B972,5940.9,863.56,0,0.0,0.0,501,1,0,0,...,11,14,17,0,0,柴,17,0,1009.953,0.00
4,00519F,5857.1,1868.83,0,0.0,0.0,k2,1,0,4,...,18,23,30,0,0,混,30,0,1757.130,0.00
5,00568F,6683.4,1833.99,5,0.0,0.0,k2,1,0,2,...,18,23,30,0,0,混,30,0,2005.020,0.00
6,01100F,5768.6,1560.72,0,0.0,0.0,k2,1,0,3,...,18,23,30,0,0,混,30,0,1730.580,0.00
7,01110F,4612.8,1499.16,0,0.0,0.0,k2,1,0,1,...,18,23,30,0,0,混,30,0,1383.840,0.00
8,01153F,7398.3,2322.19,0,0.0,0.0,k2,1,0,0,...,18,23,30,0,0,混,30,0,2219.490,0.00
9,01180F,7033.9,2066.94,0,0.0,0.0,k2,1,0,1,...,18,23,30,0,0,混,30,0,2110.170,0.00


In [10]:
result_data.iloc[0]

car_id                 B723
mileage              6273.6
oil_cost             995.71
maintain                  5
follow                    0
inspection                0
route                   501
team_x                    1
fix_days                  0
stop_days                 2
work_days                29
engage_mileage            0
public_mileage            0
shunt_mileage         371.6
fault_times               0
fault_minutes             0
team_y                    1
sub_car_id             B723
cartype_id               11
id                       11
target_value1            14
target_value2            17
target_value3             0
target_value4             0
power_type                柴
target_oil_cost          17
target_elc_cost           0
total_oil_target    1066.51
total_elc_target          0
Name: 0, dtype: object

In [13]:
import imp
imp.reload(fd)

<module 'feedback_data' from 'C:\\Users\\ajbzx\\bus_sys_dst\\feedback_data.py'>

In [14]:
from xlwt import Workbook
import feedback_data as fd
teams=result_data["team_x"].drop_duplicates().values
for team in teams:
    newWb = Workbook()
    routes=result_data[result_data["team_x"]==team]["route"].drop_duplicates().values
    for route in routes:
        ws = newWb.add_sheet(str(route)+"路统计表")
        fd.write_car_oil_cost_sum(ws,date,result_data[result_data["route"]==route],team=team,route=route)
        ws=newWb.add_sheet(str(route)+"路汇总表")
        fd.write_car_oil_cost_detail(ws,date,result_data[result_data["route"]==route],team=team,route=route)
    newWb.save("output\\"+team+".xls")

In [76]:
from tools.filetool import createdir
newWb = Workbook()
ws = newWb.add_sheet("反馈汇总表")
fd.write_monthy_feedback_sum_table(ws, date,result_data)
ws_ = newWb.add_sheet("单车汇总表")
fd.write_monthy_feedback_detail_table(ws_, date,result_data)
createdir("output\\2019\\反馈数据\\8月\\")
newWb.save("output\\2019\\反馈数据\\8月\\8月反馈汇总表.xls")


In [95]:
import imp
imp.reload(fd)
newWb = Workbook()
ws = newWb.add_sheet("公里汇总表")
fd.write_monthy_mileage_table(ws,date,result_data)
newWb.save("output\\2019\\反馈数据\\8月\\8月公里反馈.xls")

<module 'feedback_data' from 'C:\\Users\\ajbzx\\bus_sys_dst\\feedback_data.py'>